<a href="https://colab.research.google.com/github/Mind-Interfaces/Human-Emulation-System/blob/main/Human_Emulation_System_(Clarifai_%2B_Llama_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%time
import os
import time
start_time = time.time()

try:
    import clarifai_grpc
    print ('GRPC already installed.')
except ImportError:
    os.system('pip install clarifai-grpc')

GRPC already installed.
CPU times: user 1.53 ms, sys: 0 ns, total: 1.53 ms
Wall time: 1.54 ms


In [8]:
%%time
try:
    import gradio as gr
    print ('Gradio already installed.')
except ImportError:
    os.system('pip install gradio')

Gradio already installed.
CPU times: user 1.99 ms, sys: 2 µs, total: 1.99 ms
Wall time: 2.06 ms


In [9]:
%%time
# Human Emulation System (Edge Edition) (HES-EDGE.py)

# Import Gradio
from gradio import Interface
from gradio.components import Textbox
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel

CPU times: user 33 µs, sys: 4 µs, total: 37 µs
Wall time: 41 µs


In [10]:
class HumanEmulationSystem:
    def __init__(self):
        # Define configuration settings.
        self.clarifai_pat = 'YOUR_PAT_HERE' # API key for Clarifai.

        # Configure log.
        self.chat_history = ""

        # Initialize Clarifai API.
        self.channel = ClarifaiChannel.get_grpc_channel()
        self.stub = service_pb2_grpc.V2Stub(self.channel)
        self.metadata = (('authorization', 'Key ' + self.clarifai_pat),)

    def get_clarifai_response(self, model_id, user_id, app_id, text_input):
        """Fetch response from Clarifai model."""
        try:
            user_data_object = resources_pb2.UserAppIDSet(user_id=user_id, app_id=app_id)
            post_model_outputs_response = self.stub.PostModelOutputs(
                service_pb2.PostModelOutputsRequest(
                    user_app_id=user_data_object,
                    model_id=model_id,
                    inputs=[
                        resources_pb2.Input(
                            data=resources_pb2.Data(
                                text=resources_pb2.Text(
                                    raw=text_input
                                )
                            )
                        )
                    ]
                ),
                metadata=self.metadata
            )
            if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
                raise Exception(f"Post model outputs failed, status: {post_model_outputs_response.status.description}")

            return post_model_outputs_response.outputs[0].data.text.raw
        except Exception as e:
            return None

    def generate_response(self, text_input):
        """Generate a text response."""
        # Use CodeLlama as logic model.
        logic_response = self.get_clarifai_response('CodeLlama-7B-Instruct-GPTQ', 'clarifai', 'ml', text_input)

        # Use Orca as reason model.
        reason_response = self.get_clarifai_response('orca_mini_v3_13B-GPTQ', 'clarifai', 'ml', text_input)

        # META to Combine responses.
        combined_response = f"(Logical Response: {logic_response})\n(Creative Response: {reason_response})"
        combined_input = f"Instruction: {text_input}]\n[Integrate both of the responses into a unified response:"
        combined_input += f"\n{combined_response}]\n[Combine the two responses into a single, cohesive response:]\n"

        # Use Llama chat model to recompile response.
        # response = self.get_clarifai_response('llama2-70b-chat', 'meta', 'Llama-2', combined_input)
        response = self.get_clarifai_response('llama2-13b-chat', 'meta', 'Llama-2', combined_input)

        return response

In [11]:
# Initialize the Human Emulation System.
HES = HumanEmulationSystem()

# Create and launch Gradio Interface.
interface = Interface(
    fn=HES.generate_response,
    inputs=Textbox(lines=10, placeholder="Enter Text Here..."),
    outputs="text"
)

if __name__ == '__main__':
    interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6562cbe7a818363de4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
